In [8]:
import tensorflow as tf
import numpy as np
from selenium import webdriver
import os

In [67]:
DRIVER = 'geckodriver'
driver = webdriver.Firefox()
driver.implicitly_wait(3)
driver.get('https://play2048.co')

In [55]:
def get_board(driver):
    tiles = driver.find_element_by_class_name('tile-container')
    values = [0]*16
    for tile in tiles.find_elements_by_tag_name('div'):
        className = tile.get_attribute('class')
        # lazy method of avoiding the tile-inner class
        if len(className) > 20:
            x = int(className[26]) - 1
            y = int(className[28]) - 1
            index = 4*y + x
            values[index] = int(tile.text)
    return values

In [344]:
np.array(img)[0,0]

array([250, 248, 239, 255], dtype=uint8)

# Get an array of box values

In [406]:
#get pixel data and search for color of bounding box
values = [0]*16
index = 0
bounding = np.array(img)
for row in range(0, 4):
    for col in range(0, 4):
        if np.array_equal(bounding[row*160+30, col*160+30],
                          np.array([205, 193, 180, 255])):
            values[index] = 0
        else:
            #TODO: add in Rekognition
            values[index] = int(readValues[0][0])
            del readValues[0]
        index += 1 

IndexError: list index out of range

In [353]:
values

[0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 4, 2]

In [48]:
def score_state(state):
    nonZeros = 0
    for num in state:
        if num != 0:
            nonZeros += 1
    score = sum(state)/nonZeros
    return score

In [57]:
def get_move(values):
    lstate = values
    changing = True
    while changing:
        changing = False
        for i in range(0, len(values) -1):
            if i in [3,7,11,15]:
                pass
            elif lstate[i] == 0 and lstate[i+1] != 0:
                lstate[i] = lstate[i + 1]
                lstate[i + 1] = 0
                changing = True
            elif lstate[i] != 0 and lstate[i+1] == lstate[i]:
                lstate[i] *= 2
                lstate[i+1] = 0
                changing = True

    rstate = values
    changing = True
    while changing:
        changing = False
        for i in range(len(values)-1, -1, -1):
            if i % 4 == 0:
                pass
            elif rstate[i] == 0 and rstate[i-1] != 0:
                rstate[i] = rstate[i-1]
                rstate[i-1] = 0
                changing = True
            elif rstate[i] != 0 and rstate[i-1] == rstate[i]:
                rstate[i] *= 2
                rstate[i-1] = 0
                changing = True
    ustate = values
    changing = True
    while changing:
        changing = False
        for i in range(0, 4):
            for j in range(0,3):
                if ustate[j*4+i] == 0 and ustate[(j+1)*4+i] != 0:
                    ustate[(j)*4+i] = ustate[(j+1)*4+i]
                    ustate[(j+1)*4+i] = 0
                    changing = True
                elif ustate[j*4+i] != 0 and ustate[(j+1)*4+i] == ustate[j*4+i]:
                    print(j*4+i)
                    ustate[(j)*4+i] *= 2
                    ustate[(j+1)*4+i] = 0
                    changing = True
    dstate = values
    changing = True
    while changing:
        changing = False
        for i in range(3, -1, -1):
            for j in range(3,0,-1):
                if dstate[j*4+i] == 0 and dstate[(j-1)*4+i] != 0:
                    dstate[(j)*4+i] = dstate[(j-1)*4+i]
                    dstate[(j-1)*4+i] = 0
                    changing = True
                elif dstate[j*4+i] != 0 and dstate[(j-1)*4+i] == dstate[j*4+i]:
                    dstate[(j)*4+i] *= 2
                    dstate[(j-1)*4+i] = 0
                    changing = True
    scores = [score_state(x) for x in [lstate, rstate, ustate, dstate]]
    return np.argmax(scores)

In [72]:
import time

In [53]:
e.send_keys(u'\ue015')

In [74]:
e = driver.find_element_by_tag_name('body')
for i in range(1000):
    values = get_board(driver)
    move = get_move(values)
    if move == 0:
        e.send_keys(u'\ue012')
    elif move == 1:
        e.send_keys(u'\ue014')
    elif move == 2:
        e.send_keys(u'\ue013')
    elif move == 3:
        e.send_keys(u'\ue015')
    time.sleep(.3)


6
7
2
6
2
6
2
2
3
6
7
2
6
7
2
6
7
2
2
10
11
10
3
6
3
7
3
3
3
3
3
3
3
3
3
3
3


KeyboardInterrupt: 

[2, 2, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]